In [1]:
import sys
sys.path.append('/thesis')

In [2]:
import json
import pickle
# import logging
# import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from helper.character_encoder import DictionaryCharacterEncoder
from helper.prediction import predict_sequence

from implementation.seq2seq.transformer.utils import masked_loss, masked_accuracy
from implementation.seq2seq.transformer.keras_nlp import prepare_asymmetric_batches
from implementation.seq2seq.transformer.keras_nlp import construct_asymmetric_model_w_teacher_forcing

import tensorflow as tf
import keras_nlp

tf.config.list_physical_devices('GPU')

2023-05-28 10:18:25.792478: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-28 10:18:25.819618: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 10:18:26.972019: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 10:18:26.975928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bu

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Input file paths
# jrc_file = '/thesis/data/jrc_person_pairs.csv'
wikidata_file = '/thesis/data/wikidata_person_variant_list_norm.csv'

model_serialization_path = '/thesis/models/transformer_wikidata_variant_list_1'

## Loading & Preprocessing

In [4]:
# Preprocessing
MAX_ENCODER_SEQ_LENGTH = 30
MAX_DECODER_SEQ_LENGTH = 120
NUM_SAMPLES = 500000
VALIDATION_SPLIT = 0.3
RANDOM_STATE = 1010
BATCH_SIZE = 64

# Model
EMBEDDING_DIM = 128
INTERMEDIATE_DIM = 512
NUM_ENCODER_HEADS = 8
NUM_DECODER_HEADS = 8
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 6

DROPOUT = 0.2

# Training
EPOCHS = 40
CHECKPOINT_FREQ = 5

In [5]:
# pairs_df = pd.read_csv(jrc_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of JRC pairs:', len(pairs_df))
# pairs_df2 = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df2 = pairs_df2[(pairs_df2['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df2['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of Wikidata pairs:', len(pairs_df2), '\n')
# pairs_df = pd.concat([pairs_df, pairs_df2]).sample(frac=1, random_state=RANDOM_STATE)

pairs_df = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_ENCODER_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_DECODER_SEQ_LENGTH)]
pairs_df

,input,target
2,vasilij alexejevic nebenzja,wassili alexejewitsch nebensja
3,vasilij nebenzia,vassili nebenzia#vasily nebenzya
4,vasily nebenzya,vassili nebenzia#vasilij nebenzia
5,vassili nebenzia,vasilij nebenzia#vasily nebenzya
6,wassili alexejewitsch nebensja,vasilij alexejevic nebenzja
...,...,...
2230799,william chappell mlajsi,"william chappell, jr."
2230800,"william chappell, jr.",william chappell mlajsi
2230809,jean-laurent cochet,jean laurent cochet
2230813,buckwheat zydeco,stanley dural


In [6]:
encoder_dce = DictionaryCharacterEncoder(max_seq_length=MAX_ENCODER_SEQ_LENGTH+2)
decoder_dce = DictionaryCharacterEncoder(charset='simple_with_sep', max_seq_length=MAX_DECODER_SEQ_LENGTH+2)

train_batches, val_batches = prepare_asymmetric_batches(pairs_df, encoder_dce, decoder_dce, NUM_SAMPLES, VALIDATION_SPLIT, BATCH_SIZE, RANDOM_STATE)

2023-05-28 10:18:41.215420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 10:18:41.215815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 10:18:41.216027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 10:18:41.676617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 10:18:41.676835: I tensorflow/compile

In [7]:
for inputs, targets in train_batches.take(1):
    print(f'Encoder inputs shape: {inputs[0].shape}')
    print(f'Decoder inputs shape: {inputs[1].shape}')
    print(f'Targets shape: {targets.shape}')

Encoder inputs shape: (64, 31)
Decoder inputs shape: (64, 119)
Targets shape: (64, 119)


2023-05-28 10:19:04.051745: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [350000]
	 [[{{node Placeholder/_0}}]]
2023-05-28 10:19:04.051984: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [350000]
	 [[{{node Placeholder/_0}}]]


## Model

In [8]:
transformer = construct_asymmetric_model_w_teacher_forcing(
    num_encoder_layers=NUM_ENCODER_LAYERS, 
    num_decoder_layers=NUM_DECODER_LAYERS, 
    encoder_unique_tokens=len(encoder_dce.charset),
    decoder_unique_tokens=len(decoder_dce.charset),
    max_encoder_seq_length=MAX_ENCODER_SEQ_LENGTH,
    max_decoder_seq_length=MAX_DECODER_SEQ_LENGTH,
    embedding_dim=EMBEDDING_DIM,
    intermediate_dim=INTERMEDIATE_DIM,
    encoder_heads=NUM_ENCODER_HEADS,
    decoder_heads=NUM_DECODER_HEADS,
    dropout=DROPOUT
)

In [9]:
transformer.summary()
# transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.compile(
    loss=masked_loss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[masked_accuracy])

Model: "transformer_w_tf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 128)      4352        ['encoder_inputs[0][0]']         
                                                                                                  
 position_embedding (PositionEm  (None, 32, 128)     4096        ['embedding[0][0]']              
 bedding)                                                                                         
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 32, 128)     0           ['embedding[0][0]'

## Training

In [10]:
steps_per_epoch = len(train_batches)

checkpoint_path = f'{model_serialization_path}/checkpoints/' + 'weights-{epoch:03d}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, 
                monitor='val_masked_accuracy', 
                save_weights_only=True,
                save_freq=int(steps_per_epoch * CHECKPOINT_FREQ), 
                verbose=1)

In [11]:
history = transformer.fit(train_batches, epochs=EPOCHS, validation_data=val_batches, callbacks=[checkpoint])

Epoch 1/40


2023-05-28 10:19:05.692410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [350000]
	 [[{{node Placeholder/_0}}]]
2023-05-28 10:19:05.692615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [350000]
	 [[{{node Placeholder/_0}}]]
2023-05-28 10:19:14.706387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-28 10:19:15.022771: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x6abca60 initialized for platfor

5469/5469 [==============================] - ETA: 0s - loss: 0.9468 - masked_accuracy: 0.7330

2023-05-28 10:27:12.716479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [150000]
	 [[{{node Placeholder/_1}}]]
2023-05-28 10:27:12.716773: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [150000]
	 [[{{node Placeholder/_0}}]]


5469/5469 [==============================] - 568s 100ms/step - loss: 0.9468 - masked_accuracy: 0.7330 - val_loss: 0.6070 - val_masked_accuracy: 0.8202
Epoch 2/40
5469/5469 [==============================] - 547s 100ms/step - loss: 0.6273 - masked_accuracy: 0.8148 - val_loss: 0.5476 - val_masked_accuracy: 0.8339
Epoch 3/40
5469/5469 [==============================] - 547s 100ms/step - loss: 0.5824 - masked_accuracy: 0.8254 - val_loss: 0.5194 - val_masked_accuracy: 0.8411
Epoch 4/40
5469/5469 [==============================] - 546s 100ms/step - loss: 0.5574 - masked_accuracy: 0.8314 - val_loss: 0.5005 - val_masked_accuracy: 0.8461
Epoch 5/40
5468/5469 [============================>.] - ETA: 0s - loss: 0.5408 - masked_accuracy: 0.8354
Epoch 5: saving model to /thesis/models/transformer_wikidata_variant_list_1/checkpoints/weights-005
5469/5469 [==============================] - 547s 100ms/step - loss: 0.5409 - masked_accuracy: 0.8354 - val_loss: 0.4883 - val_masked_accuracy: 0.8486
Epoch 6

In [12]:
transformer.save(f'{model_serialization_path}/')

2023-05-28 16:24:52.362643: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,128]
	 [[{{node inputs}}]]
2023-05-28 16:24:52.372394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,128]
	 [[{{node decoder_sequence}}]]
2023-05-28 16:24:52.382666: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,128]
	 

INFO:tensorflow:Assets written to: /thesis/models/transformer_wikidata_variant_list_1/assets


INFO:tensorflow:Assets written to: /thesis/models/transformer_wikidata_variant_list_1/assets


In [13]:
with open(model_serialization_path + '/train_history.p', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

train_config = {
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'num_samples': NUM_SAMPLES,
    'max_seq_length': (MAX_ENCODER_SEQ_LENGTH, MAX_DECODER_SEQ_LENGTH),
    'random_state': RANDOM_STATE,
    'validation_split': VALIDATION_SPLIT,
    'encoder_layers': NUM_ENCODER_LAYERS,
    'decoder_layers': NUM_DECODER_LAYERS,
    'encoder_heads': NUM_ENCODER_HEADS,
    'decoder_heads': NUM_DECODER_HEADS,
    'dropout': DROPOUT,
    'embedding_dim': EMBEDDING_DIM,
    'intermediate_dim': INTERMEDIATE_DIM
}

with open(model_serialization_path + '/config.p', 'wb') as file_pi:
    pickle.dump(train_config, file_pi)

In [14]:
transformer = tf.keras.models.load_model(f'{model_serialization_path}/', custom_objects={'masked_loss': masked_loss, 'masked_accuracy': masked_accuracy})

## Prediction

In [ ]:
def decode_sequences(input_sentence):
     # Tokenize the encoder input.
    encoder_input_tokens = tf.keras.utils.pad_sequences(
        encoder_dce.to_ids([input_sentence], insert_markers=True), 
        padding='post', 
        maxlen=MAX_ENCODER_SEQ_LENGTH+2
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return transformer([encoder_input_tokens, decoder_input_tokens])[:, -1, :]


    prompt = tf.fill((1, 1), decoder_dce.char_index['\t'])
    generated_tokens = keras_nlp.utils.top_p_search(
        token_probability_fn,
        prompt,
        p=0.1,
        max_length=MAX_DECODER_SEQ_LENGTH,
        end_token_id=decoder_dce.char_index['\n'],
    )
    generated_sentences = ''.join([decoder_dce.inverse_char_index[tkn] for tkn in generated_tokens.numpy()[0]])
    return generated_sentences.strip()

names = [
    'samuel meyer',
    'dmitry medvedev',
    'paulo ricardo',
    'zouheir al qaissi',
    'tarek al bichri',
    'thorsten brotzmann'
]

for s in names:
    translated = decode_sequences(s)
    print(s)
    print(translated)
    print()


samuel meyer
samuel meyer#samuel meyer

dmitry medvedev
dmitrij medvedev#dmitrij medvedev#dmitrij medvedev#dmitrij medvedev#dmitrij medvedev

paulo ricardo
paulus ricardus

